In [2]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

# Alternatives matrix and data
matrix = np.array([
    [0.087, 0.084, 0.115, 0.111],
    [0.066, 0.079, 0.111, 0.102],
    [0.044, 0.086, 0.085, 0.088],
    [0.088, 0.086, 0.244, 0.078],
    [0.285, 0.063, 0.122, 0.068],
    [0.040, 0.134, 0.018, 0.144],
    [0.030, 0.096, 0.020, 0.148],
    [0.039, 0.130, 0.112, 0.105],
    [0.109, 0.103, 0.124, 0.089], 
    [0.014, 0.139, 0.040, 0.069]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# Normalize matrix
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

norm_matrix = normalize_matrix(matrix, criteria_types)
df_norm = pd.DataFrame(norm_matrix, columns=criteria, index=sites)

# Criterion-wise rankings using scipy
print("\nRanking of sites for each criterion:")
criterion_ranks = pd.DataFrame(index=sites)

for i, c in enumerate(criteria):
    col = norm_matrix[:, i]
    if criteria_types[i] == 'max':
        ranks = rankdata(-col, method='min')  # Descending rank
    else:
        ranks = rankdata(col, method='min')   # Ascending rank
    criterion_ranks[c] = ranks

print(criterion_ranks)

# AHP weighted score and overall ranking
overall_scores = np.dot(norm_matrix, weights)
overall_ranks = rankdata(-overall_scores, method='min')  # Higher score = better

df_result = pd.DataFrame({
    'AHP Score': overall_scores,
    'Rank': overall_ranks
}, index=sites).sort_values('Rank')

print("\nOverall AHP Scores and Ranking:")
print(df_result)



Ranking of sites for each criterion:
        C1  C2  C3  C4
Site1    4   8   4   3
Site2    5   9   6   5
Site3    6   6   7   7
Site4    3   6   1   8
Site5    1  10   3  10
Site6    7   2  10   2
Site7    9   5   9   1
Site8    8   3   5   4
Site9    2   4   2   6
Site10  10   1   8   9

Overall AHP Scores and Ranking:
        AHP Score  Rank
Site4    0.607781     1
Site8    0.504332     2
Site9    0.450839     3
Site5    0.377254     4
Site1    0.366024     5
Site6    0.364723     6
Site10   0.342170     7
Site2    0.317067     8
Site3    0.264325     9
Site7    0.218571    10
